In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import sys, os

print(f"Installing packages into environment {sys.executable}")

Installing packages into environment c:\anaconda3\envs\woodsidepoc\python.exe


In [3]:
!{sys.executable} -m pip install azure-search-documents==11.4.0b8
!{sys.executable} -m pip install azure-identity
!{sys.executable} -m pip install python-dotenv


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

True

In [6]:
from azure.core.credentials import AzureKeyCredential  
# Configure connection
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
key_credential = AzureKeyCredential(os.getenv('AZURE_SEARCH_ADMIN_KEY'))

In [7]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient 
 
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=key_credential)

In [ ]:
#indexes = index_client.list_indexes()

#for index in indexes:
#    print(index.name)

In [19]:
from azure.search.documents.indexes.models import ( 
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    SearchIndex,
    BM25Similarity,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings
)

In [20]:
index_name="sharepoint-demo-index-51"

In [21]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False, filterable=True, retrievable=True, sortable=True, facetable=True),
    SearchableField(name="metadata_spo_item_name", type=SearchFieldDataType.String),
    SimpleField(name="metadata_spo_item_path", type=SearchFieldDataType.String, key=False, searchable=False, filterable=False, retrievable=True, sortable=False, facetable=False),
    SimpleField(name="metadata_spo_item_content_type", type=SearchFieldDataType.String, key=False, searchable=False, filterable=True, retrievable=True, sortable=False, facetable=True),
    SimpleField(name="metadata_spo_item_last_modified", type=SearchFieldDataType.DateTimeOffset, key=False, searchable=False, filterable=False, retrievable=True, sortable=True, facetable=False),
    SimpleField(name="metadata_spo_item_size", type=SearchFieldDataType.Int64, key=False, searchable=False, filterable=False, retrievable=True, sortable=False, facetable=False),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SimpleField(name="metadata_spo_item_weburi", type=SearchFieldDataType.String, searchable=False, filterable=False, retrievable=True, sortable=False, facetable=False),
    SearchField(name="textItems", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=False, sortable=False, facetable=False),
]

In [22]:
index = SearchIndex(name=index_name, fields=fields, similarity=BM25Similarity())

In [23]:
sharepoint_demo_index_51 = index_client.create_index(index)
print(f"Created index {sharepoint_demo_index_51.name}")

HttpResponseError: (ResourceNameAlreadyInUse) Cannot create index 'sharepoint-demo-index-51' because it already exists.
Code: ResourceNameAlreadyInUse
Message: Cannot create index 'sharepoint-demo-index-51' because it already exists.
Exception Details:	(CannotCreateExistingIndex) Cannot create index 'sharepoint-demo-index-51' because it already exists.
	Code: CannotCreateExistingIndex
	Message: Cannot create index 'sharepoint-demo-index-51' because it already exists.

In [24]:
index_name="sharepoint-demo-index-53"

In [25]:
vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="page-vector-config",
            kind="hnsw",
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric="cosine"
            )
        )
    ]
)

In [26]:
semantic_config = SemanticConfiguration(
    name="semantic-config-1",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="metadata_spo_item_name"),
        prioritized_keywords_fields=[SemanticField(field_name="keyPhrases")],
        prioritized_content_fields=[SemanticField(field_name="page")]
    )
)

semantic_settings = SemanticSettings(configurations=[semantic_config])

In [27]:
fields = [
    SimpleField(name="AzureSearch_DocumentKey", type=SearchFieldDataType.String, key=True, searchable=False, filterable=True, retrievable=True, sortable=True, facetable=True),
    SearchableField(name="metadata_spo_item_name", type=SearchFieldDataType.String),
    SimpleField(name="metadata_spo_item_content_type", type=SearchFieldDataType.String, key=False, searchable=False, filterable=True, retrievable=True, sortable=False, facetable=True),
    SimpleField(name="metadata_spo_item_last_modified", type=SearchFieldDataType.DateTimeOffset, key=False, searchable=False, filterable=False, retrievable=True, sortable=True, facetable=False),
    SimpleField(name="metadata_spo_item_size", type=SearchFieldDataType.Int64, key=False, searchable=False, filterable=False, retrievable=True, sortable=False, facetable=False),
    SimpleField(name="metadata_spo_item_weburi", type=SearchFieldDataType.String, searchable=False, filterable=False, retrievable=True, sortable=False, facetable=False),
    SearchableField(name="page", type=SearchFieldDataType.String),
    SearchField(name="organizations", type=SearchFieldDataType.Collection(SearchFieldDataType.String)),
    SearchField(name="persons", type=SearchFieldDataType.Collection(SearchFieldDataType.String)),
    SearchField(name="locations", type=SearchFieldDataType.Collection(SearchFieldDataType.String)),
    SearchField(name="keyPhrases", type=SearchFieldDataType.Collection(SearchFieldDataType.String)),
    SearchField(name="ocrText", type=SearchFieldDataType.Collection(SearchFieldDataType.String)),
    SearchField(name="pageVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), filterable=False, sortable=False, facetable=False, vector_search_dimensions=1536, vector_search_configuration="page-vector-config"),
]

In [28]:
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_settings=semantic_settings, similarity=BM25Similarity())

In [29]:
sharepoint_demo_index_53 = index_client.create_index(index)
print(f"Created index {sharepoint_demo_index_53.name}")

Created index sharepoint-demo-index-53
